# Agents for Amazon Bedrock - create agent

This notebook provides sample code for building an Agent for Amazon Bedrock that has an Action Group attached to it.

### Use Case
We will create a restaurant assistant that allows customers to create, delete or get reservation information. The architecture looks as following:

![Agent Architecture](./images/agent-architecture_1.png)

### Notebook Walk-through

In this notebook we will:
- Choose our Agent's underline foundation model
- Create a dynamoDB table to store the reservation details
- Create a lambda function that handles the restaurant bookings
- Create an agent
- Create an action group and associate it with the agent
- Test the agent invocation


### Next Steps: 
In the next lab, we will add a Knowledge Base to our agent and work with Prompt Attributes to provide extra information to an agent invocation call


### Pre-requisites
This notebook requires permissions to:
- create and delete Amazon IAM roles
- create lambda functions
- create dynamoDB tables
- access Amazon Bedrock

If running on SageMaker Studio, you should add the following managed policies to your role:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess
- AmazonDynamoDBFullAccess

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure to enable `Anthropic Claude 3 Sonnet` model access in Amazon Bedrock Console, as the notebook will use Anthropic Claude 3 Sonnet model for testing the agent once its created.
</div>


## Setup
Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries are installed

In [ ]:
!pip install --upgrade -q -r requirements.txt

Let's now import the necessary libraries and initiate the required boto3 clients

In [ ]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid

from agent import create_agent_role, create_lambda_role
from agent import create_dynamodb, create_lambda, invoke_agent_helper

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

### Setting up Agent's information

We will now set the variables that define our agent:

- **agent_name**: provides the name of the agent to be created, in this case `booking-agent`
- **agent_description**: the description of the agent used to display the agents list on the console. This description is **not** part of the agent's prompts
- **agent_instruction**: the instructions of what the agent should and should not do. This description is part of the agent's prompt and is used during the agent's invocation
- **agent_action_group_name**: the action group name used on the definition of the agent's action, in this case `TableBookingsActionGroup`.
- **agent_action_group_description:**: the description of the action group name used on the UI to list the action groups. This description is **not** used by the agent's prompts

In [ ]:
suffix = f"{region}-{account_id}"
agent_name = 'booking-agent'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of a restaurants table bookings"
agent_instruction = """
You are a restaurant agent, helping clients retrieve information from their booking,
create a new booking or delete an existing booking
"""

agent_action_group_description = """
Actions for getting table booking information, create a new booking or delete an existing booking"""

agent_action_group_name = "TableBookingsActionGroup"

### Select Foundation Model
Use this dropdown menu to select the underline model of your agent. You can find more information about the supported foundation models [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-supported.html)

In [ ]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        ('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
        ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

Let's confirm that the model has been selected correctly

In [ ]:
agent_foundation_model = agent_foundation_model_selector.value
agent_foundation_model

### Creating DynamoDB table

Let's now create an [Amazon DynamoDB](https://aws.amazon.com/dynamodb/) table called `restaurant_bookings`. This table will store information about the reservations, including a `booking_id`, reservation `date`, the `name` of the person doing the reservation, the `hour` of the reservation and the number of guests as `num_guests`. To do so, we use the `create_dynamodb` function from the `agent.py` file. This function will support the creation of the table and its requirements (IAM roles and permissions).

In [ ]:
table_name = 'restaurant_bookings'
create_dynamodb(table_name)

### Creating Lambda Function

Next we will create the [AWS Lambda](https://aws.amazon.com/lambda/) function that executes the actions for our agent. This lambda function will have 3 actions:
* ```get_booking_details(booking_id)```: returns the details of a booking based on the booking id
* ```create_booking(date, name, hour, num_guests)```: creates a new booking for the restaurant
* ```delete_booking(booking_id)```: deletes an existent booking based on the booking id


The `lambda_handler` receives the `event` from the agent and the `event` contains information about the `function` to be executed and its `parameters`. 

A `functionResponse` is returned by the lambda function with the response body having a `TEXT` field.

You can find more information on how to set your agent lambda function [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html).

Let's first write the code of the lambda function to the `lambda_function.py` file

In [ ]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('restaurant_bookings')

def get_named_parameter(event, name):
    """
    Get a parameter from the lambda event
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_booking_details(booking_id):
    """
    Retrieve details of a restaurant booking
    
    Args:
        booking_id (string): The ID of the booking to retrieve
    """
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No booking found with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}


def create_booking(date, name, hour, num_guests):
    """
    Create a new restaurant booking
    
    Args:
        date (string): The date of the booking
        name (string): Name to idenfity your reservation
        hour (string): The hour of the booking
        num_guests (integer): The number of guests for the booking
    """
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        return {'error': str(e)}


def delete_booking(booking_id):
    """
    Delete an existing restaurant booking
    
    Args:
        booking_id (str): The ID of the booking to delete
    """
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'Failed to delete booking with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}
    

def lambda_handler(event, context):
    # get the action group used during the invocation of the lambda function
    actionGroup = event.get('actionGroup', '')
    
    # name of the function that should be invoked
    function = event.get('function', '')
    
    # parameters to invoke function with
    parameters = event.get('parameters', [])

    if function == 'get_booking_details':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    elif function == 'create_booking':
        date = get_named_parameter(event, "date")
        name = get_named_parameter(event, "name")
        hour = get_named_parameter(event, "hour")
        num_guests = get_named_parameter(event, "num_guests")

        if date and hour and num_guests:
            response = str(create_booking(date, name, hour, num_guests))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'delete_booking':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Next we create the function requirements for IAM role and policies using the support function `create_lambda_role` and create the lambda using the support function `create_lambda` both from the `agent.py` file

In [ ]:
lambda_iam_role = create_lambda_role(agent_name, table_name)

In [ ]:
lambda_function_name = f'{agent_name}-lambda'

In [ ]:
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

### Creating Agent

Now that we have created the dynamoDB table and lambda function, let's create our Agent. 

To do so, we first need to create an agent role and its required policies. Let's do so using the `create_agent_role` function from the `agent.py` file.

In [ ]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

In [ ]:
agent_role

With the Agent IAM role created, we can now use the boto3 function [`create_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html) to create our agent. 

On the agent creation, all you need to provide is the agent name, foundation model and instruction. We will associate an action group to the agent once it has been created

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

Now that our agent has been created, we will retrieve the `agentId`. It will be used to associate the action group to the agent in our next step.

In [ ]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

#### Create Agent Action Group

now that we have created the agent, let's create an [Action Group](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html) and associate with the agent. The action group will allow our agent to execute the booking tasks. To do so, we will "inform" our agent about the existent functionalities using a [function schema](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html) defined in `JSON` format.

The function schema requires the function `name`, `description` and `parameters` to be provided. Each parameter has a parameter name, description, type and a boolean flag indicating if the parameter is required.

Let's define the functions `JSON` as `agent_functions`

In [ ]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking in the format HH:MM",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

Now we can use the [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function from the boto3 SDK to create the action group

In [ ]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
agent_action_group_response

#### Allowing bedrock to invoke lambda function

The last requirement is to add the [resource-based policy](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda) to allow bedrock to invoke the action group lambda function.

In [ ]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

#### Preparing agent

Before invoking the agent we need to prepare it. Preparing your agent will package all its components, including the security configurations. It will bring the agent into a state where it can be tested in runtime. We will use the [`prepare_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/prepare_agent.html) function from the boto3 sdk to prepare our agent.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

### Invoking Agent

Now that our Agent is ready to be used, let's test it. To do so we will use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

To invoke an agent, you have to refer to its alias. You can create a new alias, or you can use the test alias to invoke your `DRAFT` agent. The test alias used to invoke the draft agent is `TSTALIASID` and it will work with any agent. 


We will use the support function called `invoke_agent_helper` from the `agents.py` support file to allow us to invoke the agent with or without trace enabled and with or without session state. We will getinto more details about those concepts in the `03_invoke_agent.ipynb` notebook.

In [ ]:
alias_id = 'TSTALIASID'

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "Hi, I am Anna. I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

### Next Steps

Before moving to the next notebook, let's store a couple of variables to continue working the the same notebook.

Next we will update our agent to associate a knowledge base containing the menus for our restaurant. We will then test the agent invocations and clean up all the created resources.

In [ ]:
%store agent_id
%store agent_role
%store lambda_iam_role
%store agent_name
%store suffix
%store region
%store agent_foundation_model
%store account_id
%store alias_id
%store table_name
%store lambda_function
%store lambda_function_name
%store agent_action_group_response
%store agent_functions